# Hands On Python 3 - Datenaufbereitung
Dieses Notebook zeigt wie
- man Daten aus zwei csv Dateien zu einem Datensatz zusammenfügen kann
- wie man mit nicht existenten Werten (NaN - Not a Number) umgeht
- wie man Zeitreihen mit unterschiedlichen Frequenzen auf eine gemeinsame Frequenz bringen kann

## Notwendige Bibliotheken importieren

In [ ]:
# notwendige Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Daten einlesen

In [ ]:
# Daten einlesen

# Daten der 1. Quelle
rawData1 = pd.read_csv('/content/drive/My Drive/source1.csv')

# Lesen Sie die Daten der 2. Quelle ein
rawData2 = pd.read_csv('/content/drive/My Drive/source2.csv')

In [1]:
# Anzahl der Zeilen und Spalten ausgeben 

# Geben Sie Zeilen und Spalten des 1. Datensatzes von Quelle 1 aus
print("Datensatz 1 aus Quelle 1:")
print(f"Zeilen: {rawData1.shape[0]}, Spalten: {rawData1.shape[1]}")

# Geben Sie Zeilen und Spalten des 2. Datensatzes von Quelle 2 aus
print("Datensatz 2 aus Quelle 2:")
print(f"Zeilen: {rawData2.shape[0]}, Spalten: {rawData2.shape[1]}")

In [2]:
# Geben Sie die ersten Zeilen von Quelle 1 aus:
print("Erste Zeilen von Datensatz 1 aus Quelle 1:")
print(rawData1.head())

In [3]:
# Geben Sie die ersten Zeilen von Quelle 2 aus:
print("Erste Zeilen von Datensatz 2 aus Quelle 2:")
print(rawData2.head())

In [4]:
# Geben Sie die letzten Zeilen von Quelle 1 aus:
print("Letzte Zeilen von Datensatz 1 aus Quelle 1:")
print(rawData1.tail())

In [5]:
# Geben Sie die letzten Zeilen von Quelle 2 aus
print("Letzte Zeilen von Datensatz 2 aus Quelle 2:")
print(rawData2.tail())

## Daten auf fehlende Werte hin überprüfen

In [6]:
# Geben Sie sich die Anzahl der NaNs in den Spalten von Quelle 2 aus:
print("Anzahl der NaNs in den Spalten von Quelle 2:")
print(rawData2.isna().sum())

In [7]:
# Sofern der Datensatz keine NaNs erhält --> Kopie der Rohdaten namens data2 erstellen und damit direkt weiterarbeiten:
if rawData2.isna().sum().sum() == 0:
    data2 = rawData2.copy()
    print("Keine NaNs in rawData2, Kopie als data2 erstellt.")
else:
    print("rawData2 enthält NaNs.")

In [8]:
# Wie sieht es bei rawData1 aus? Geben Sie für rawData1 die Anzahl der NaNs in den Spalten aus
print("\nAnzahl der NaNs in den Spalten von Quelle 1:")
print(rawData1.isna().sum())

## Mit NaNs umgehen

### Möglichkeit 1: alle Zeilen, in denen mindestens 1x NaN vorkommt, löschen

In [ ]:
# Kopie der Rohdaten anlegen
data1_del = rawData1.copy() 

# Zeilen mit mind. 1x NaN löschen (d.h. in irgendeiner Spalte kommt ein NaN vor --> Zeile wird gelöscht)
data1_del.dropna(inplace=True)

# Überprüfen
data1_del.isna().sum()

Date         0
Energy       0
Light        0
CO2          0
Occupancy    0
dtype: int64

### Möglichkeit 2: Fehlende Werte durch interpolierte Werte ersetzen

### Linear Interpolieren

In [9]:
# Führen Sie eine lineare Interpolation durch. Erstellen Sie sich dazu zunächst data1_ip als Kopie von rawData1
data1_ip = rawData1.copy()
data1_ip.interpolate(method='linear', inplace=True)

# Geben Sie sich die Anzahl an NaN in den Spalten nach der Interpolation aus:
print(data1_ip.isna().sum())

In [12]:
# Plotten der alten Zeitreihe
plt.figure(figsize=(18, 4))
rawData1['Light'].plot(marker='o')   

In [11]:
#neue Zeitreihe mit interpolierten Werten plotten
plt.plot(data1_ip.index, data1_ip['Light'], marker='o', label='Interpolierte Zeitreihe')

Wie zufrieden sind Sie mit der Interpolation?


....

In [ ]:
# Grundsätzlich ist auch möglich mit Polynomfunktionen zu interpolieren.
# Interpolieren Sie quadratisch (method='polynomial', order=2)
data1_qp = rawData1.copy()
data1_qp.interpolate(method='polynomial', order=2, inplace=True)


# Anzahl an NaN in Spalten ausgeben
print("Anzahl der NaNs in den Spalten von data1_qp nach der quadratischen Interpolation:")
print(data1_qp.isna().sum())

Date         0
Energy       0
Light        0
CO2          0
Occupancy    0
dtype: int64

In [14]:
# Plotten Sie zum Vergleich wieder die alte und die neue Zeitreihe 
# Plot der alten Zeitreihe:
plt.figure(figsize=(18, 4))
plt.plot(rawData1.index, rawData1['Light'], marker='o', label='Original Zeitreihe')


In [15]:
# Plot der neuen Zeitreihe:
plt.plot(data1_qp.index, data1_qp['Light'], marker='o', label='Quadratisch interpolierte Zeitreihe')

Hat sich die quadratische Interpolation im Vergleich zur linearen gelohnt? Welches der beiden Interpolationsverfahren würden Sie wählen?

.......

## Zeitstempel als Index setzen
- Momentan sind die Zeilenindizes einfach durchnummeriert: 0,1,2,...
- Wenn der Zeitstempel als Index gesetzt wird, kann man einfacher arbeiten
- Dafür muss der Zeitstempel als erstes in das Format datetime gebracht werden, damit Python ihn als Zeitstempel erkennt

In [ ]:
# die Spalte Date ist noch nicht im datetime Format
data1_ip.dtypes

Date          object
Energy       float64
Light        float64
CO2          float64
Occupancy    float64
dtype: object

In [16]:
# Bringen Sie die Spalte Date ins datetime Format 
data1_ip['Date'] = pd.to_datetime(data1_ip['Date'])


In [17]:
# Indizieren Sie mit der Zeitstempel-Spalte 
data1_ip.set_index('Date', inplace=True)


# Kontrollieren Sie das durch Ausgabe der ersten Zeilen des Datensatzes. 
print("Erste Zeilen von data1_ip:")
print(data1_ip.head())

Wieso wird bei wiederholtem Ausführen obiger Zelle ein Fehler geworfen? Müssen Sie etwas am Code korrigieren?

....

In [18]:
# Indizieren Sie auch data2 mit dessen Zeitstempel


In [18]:
data2['Date'] = pd.to_datetime(data2['Date'])

In [18]:
data2.set_index('Date', inplace=True)

In [18]:
print(data2.head())

## Datensätze zu einem Zusammenfügen
- Problem: Die Datensätze haben unterschiedliche Frequenzen: Datensatz 1: 15min, Datensatz 2: stündlich
- Möglichkeit 1: Nur die Indizes nehmen, die sowohl in Datensatz 1, als auch in Datensatz 2 sind, also jede volle Stunde
- Möglichkeit 2: Den Datensatz mit der höheren Frequenz (Datensatz 1) runtersamplen, z.B. indem der stündliche Mittelwert oder die Summe über je eine Stunde gebildet wird

Frage: Worauf muss man aufpassen, wenn man Möglichkeit 2 wählt?

Antwort: 
....

### Möglichkeit 1: Nur die Indizes nehmen, die sowohl in Datensatz 1, als auch in Datensatz 2 sind

In [ ]:
# Neuen Datensatz definieren, der aus Datensatz 2 von rechts angehängt an Datensatz 1 besteht
data = pd.concat([data1_ip, data2], axis=1)

# Dort wo keine Werte für den jeweiligen Zeitstempel vorhanden sind, werden NaN eingefügt
data.head()

,Energy,Light,CO2,Occupancy,Temperature
Date,,,,,
2015-02-02 14:00:00,NaN,NaN,NaN,NaN,23.657118
2015-02-02 14:15:00,26.285636,527.283333,785.951515,1.0,NaN
2015-02-02 14:30:00,27.016313,481.697917,894.539583,1.0,NaN
2015-02-02 14:45:00,27.612107,499.415646,992.291327,1.0,NaN
2015-02-02 15:00:00,28.006019,456.436190,1051.151905,1.0,23.293950


In [20]:
# Löschen Sie nun noch die Zeilen in denen mindestens 1x NaN vorkommt:
data.dropna(inplace=True)

# Ausgabe der ersten Zeilen von data zur Kontrolle:
print("Erste Zeilen von data:")
print(data.head())

## Möglichkeit 2: 
- Datensatz 1 soll von 15minüten auf stündliche Werte runtergesampelt werden
- Für die Spalten Light und CO2 soll dazu der stündliche Mittelwert genommen werden
- Für die Spalte Energy soll dazu jeweils die Summe über die 4 Werte in einer Stunde gebildet werden
- Für die Spalte Occupancy soll einfach der Wert zur vollen Stunde genommen werden

In [21]:
# Ursprünglicher Datensatz
data1_ip.head()

NameError: ignored

In [ ]:
# Ursprünglichen Datensatz aufteilen

# nur die Spalten, die gemittelt werden sollen 
data1_mean = data1_ip.copy()
data1_mean.drop(['Energy', 'Occupancy'], axis=1, inplace=True)
data1_mean.head()

,Light,CO2
Date,,
2015-02-02 14:15:00,527.283333,785.951515
2015-02-02 14:30:00,481.697917,894.539583
2015-02-02 14:45:00,499.415646,992.291327
2015-02-02 15:00:00,456.436190,1051.151905
2015-02-02 15:15:00,464.760417,1101.779167


In [22]:
# Generieren Sie einen Datensatz data1_sum, der nur die Spalten enthält, die summiert werden sollen
data1_sum = data1_ip[['Energy']]
print(data1_sum.head())

In [23]:
# Generieren Sie einen Datensatz data1_binary, der nur die Spalten enthält, die weder gemittelt noch aufsummiert werden sollen
data1_binary = data1_ip[['Occupancy']]

In [24]:
# Bringen Sie den Datensatz data1_mean durch Mittelwertbildung auf eine höhere Frequenz (stündlich):
data1_mean_resampled = data1_mean.resample('1H').mean()

# Lassen Sie sich zur Kontrolle die ersten Zeilen ausgeben:
print(data1_mean_resampled.head())

In [25]:
# Bringen Sie data1_sum durch Summenbildung auf eine höhere Frequenz (stündlich). (statt .mean() verwenden Sie .sum())
data1_sum_resampled = data1_sum.resample('1H').sum()

# Ausgabe der ersten Zeilen zur Kontrolle:
print(data1_sum_resampled.head())

In [26]:
# Bringen Sie den Datensatz data1_binary auf eine höhere Frequenz (stündlich). 
# Dafür soll immer der Wert des Merkmals Occupancy zu vollen Stunde genommen werden, die anderen Werte sollen verworfen werden. 
data1_binary_resampled = data1_binary.resample('1H').first()

# Ausgabe der ersten Zeilen zur Kontrolle:
print(data1_binary_resampled.head())

In [ ]:
# Daten zusammenfügen
data = pd.concat([data2, data1_resampled_sum, data1_resampled_mean, data1_resampled_binary], axis=1)

data.head()

,Temperature,Energy,Light,CO2,Occupancy
Date,,,,,
2015-02-02 14:00:00,23.657118,NaN,502.798965,890.927475,1.0
2015-02-02 15:00:00,23.293950,80.914056,456.677902,1103.590327,1.0
2015-02-02 16:00:00,22.773142,113.657907,434.929193,980.338952,1.0
2015-02-02 17:00:00,22.534520,106.986065,426.717991,821.892560,1.0
2015-02-02 18:00:00,21.993372,99.873477,32.984167,719.712031,0.0


In [ ]:
# Entfernen von Zeilen mit mind. 1x NaN falls vorhanden:
data.dropna(inplace=True)

In [ ]:
# Ausgabe der ersten Zeilen von data
data.head()

,Temperature,Energy,Light,CO2,Occupancy
Date,,,,,
2015-02-02 15:00:00,23.293950,80.914056,456.677902,1103.590327,1.0
2015-02-02 16:00:00,22.773142,113.657907,434.929193,980.338952,1.0
2015-02-02 17:00:00,22.534520,106.986065,426.717991,821.892560,1.0
2015-02-02 18:00:00,21.993372,99.873477,32.984167,719.712031,0.0
2015-02-02 19:00:00,21.276331,98.368227,0.000000,585.110384,0.0


In [ ]:
# Ausgabe der Anzahl NaN in den jeweiligen Spalten

data.isna().sum()

Temperature    0
Energy         0
Light          0
CO2            0
Occupancy      0
dtype: int64

In [ ]:
an_count = data.isna().sum()
print(nan_count)